# Natural Computing - Project
#### Submission by group 25 (Chihab Amghane, Max Driessen, Jordy Naus)

The code below uses the DEAP framework (https://github.com/deap/deap), which is a very intuitive framework for evolutionary algorithms and genetic programming.

## Imports

In [26]:
# DEAP
from deap import gp, base, tools, creator, algorithms

# Data processing and plotting
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Requirements for the algorithm
from operator import attrgetter
from functools import partial

# Standard python imports
import random, pickle, math, re, os, copy
import numpy as np

import pdb
# Magic for inline plots
%matplotlib inline

## Helper functions

In [27]:
def exp(x):
    return np.exp(np.clip(x, -float('inf'), 709.))

## Global parameters

In [28]:
# Dataset parameters
DATASET = "MNIST" # choose from {"MNIST", "Fashion-MNIST"} 
N_CLASSES_TO_USE = 10 # at most 10
USE_NORMALIZATION = True
USE_ANTI_ALIASING = False

# Individual tree parameters
N_INITIAL_CONNECTIONS = 100
CONNECTIONS_CAPPED = True
MAX_CONNECTIONS = 300

# Fitness parameters
N_SAMPLES_TO_TEST = 200

# Evolution parameters
N_GENERATIONS = 500
POPULATION_SIZE = 100

TOURNAMENT_SIZE = 10
SIZE_TOURNAMENT = True
P_SMALLER_WINS = 0.65

P_ADD_CONNECTION = 0.5
P_REMOVE_CONNECTION = 0.3
P_ADD_NODE = 0.2
P_REMOVE_NODE = 0.2
P_CHANGE_OPERATOR = 0.3

# Filename parameters
RESULTS_FILENAME = "results.pkl"

## Loading preprocessed data

In [29]:
# Set the correct data filename
filename = f"{DATASET}-{N_CLASSES_TO_USE}{'-Norm' if USE_NORMALIZATION else ''}{'-AA' if USE_ANTI_ALIASING else ''}.pkl"

# If the data has not yet been preprocessed in the specified way, do so now
if not os.path.exists(os.path.join("data", filename)):
    print("Preprocessed dataset does not exist yet, creating now.")
    os.system(f"python Preprocessing.py -d {DATASET} -c {N_CLASSES_TO_USE}" + \
              f"{' --aa' if USE_ANTI_ALIASING else ''}{' --n' if USE_NORMALIZATION else ''}")

# Load the preprocessed data
with open(os.path.join("data", filename), "rb") as f:
    (X_train, Y_train), (X_test, Y_test) = pickle.load(f)

## Defining operators

In [30]:
# Define operators (with a variable number of inputs)
def linear(*args):
    return sum(args)

def inverse(*args):
    return -sum(args)

def tanh(*args):
    return math.tanh(sum(args))

def sigmoid(*args):
    return 1.0/(1.0 + exp(-sum(args)))

def step(*args):
    return float(sum(args) >= 0)

def sine(*args):
    return math.sin(sum(args))

def cosine(*args):
    return math.cos(sum(args))

def gaussian(*args):
    return 0.5*sum(args)*(1.0+math.tanh(math.sqrt(2.0/math.pi)*(sum(args) + 0.044715*sum(args)**3)))

# def gaussian(*args):
#     return exp( -(sum(args)*sum(args)) / 2.0 )
                          
def absolute(*args):
    return abs(sum(args))

In [31]:
# Define dictionary of functions for compiling
function_context = {'linear':linear, 'inverse':inverse, 'tanh':tanh, 'sigmoid':sigmoid, 'step':step, 
                    'sine':sine, 'cosine':cosine, 'gaussian':gaussian, 'absolute':absolute}

# Create lists of function and argument names
function_names = list(function_context.keys())
argument_names = [f"ARG{i}" for i in range(X_train.shape[1])]

## Defining individuals

##### Defining nodes

In [32]:
# Generic Node class
class Node:
    def __init__(self, name):
        self.name = name
        self.parents = []
        self.children = []
    
    def __str__(self):
        raise NotImplementedError("String function is only implemented for subclasses")

# Class for terminal nodes (inputs)
class TerminalNode(Node):
    def __init__(self, name):
        super().__init__(name)

    def __str__(self):
        return self.name

# Class for primitive nodes (internals + outputs)
class PrimitiveNode(Node):
    def __init__(self, name):
        super().__init__(name)

    def __str__(self):
        return f"{self.name}({', '.join([str(child) for child in self.children])})"

In [33]:
# Class for multi-output trees
class MultiClassTree:
    def __init__(self, n_inputs, n_outputs, n_initial_connections):
        # Initialize lists of input, output and internal nodes
        self.inputs = [TerminalNode(argument_names[i]) for i in range(n_inputs)]
        self.outputs = [PrimitiveNode("sigmoid") for _ in range(n_outputs)]
        self.internals = []
        
        # Add initial connections so that initial tree is valid
        self.n_connections = n_initial_connections
        for output in self.outputs:
            initial_children = random.choices(self.inputs, k=n_initial_connections)
            output.children.extend(initial_children)
            for child in initial_children:
                child.parents.append(output)

    def __str__(self):
        return f"MultiClassTree with {len(self.inputs)} inputs, " + \
                                   f"{len(self.outputs)} outputs and " + \
                                   f"{len(self.internals)} internal nodes.\n"
    
    # Retrieving output function strings
    def get_strings(self):
        try:
            return [str(output) for output in self.outputs]
        except RecursionError:
            print("Maximum recursion depth reached")
            return None

In [34]:
# Intialize the toolbox which will contain all sorts of functions for the genetic programming process
toolbox = base.Toolbox()

In [35]:
# Define classes for fitness and individuals (using DEAP's creator module)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", MultiClassTree, fitness=creator.FitnessMax)

/home/chb3k/anaconda3/envs/py37nc/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/chb3k/anaconda3/envs/py37nc/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [36]:
# Define how to initialize an individual or population
toolbox.register("individual", creator.Individual, X_train.shape[1], 
                               N_CLASSES_TO_USE, N_INITIAL_CONNECTIONS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

## Fitness function

In [37]:
# Compiling a tree into a function
def compile_multiclasstree(tree):
    strings = tree.get_strings()
    funcs = [eval(f"lambda {', '.join(argument_names)}: {string}", function_context, {}) for string in strings]
    def func(args):
        return [f(*args) for f in funcs]
    return func

In [38]:
# Add compile function to toolbox
toolbox.register("compile", compile_multiclasstree)

In [39]:
# Define fitness function   
def fitness(individual, n_samples_to_test):
    # Compile the functions corresponding to the individual
    func = toolbox.compile(individual)
    
    # Create a list of samples to test, ensuring an equal number of samples from each class
    sample_indices = []
    n_samples_per_class = int(n_samples_to_test/N_CLASSES_TO_USE)
    for c in range(N_CLASSES_TO_USE):
        c_indices = np.where(Y_train == c)[0]
        assert len(c_indices) >= n_samples_per_class, \
            f"Class {c} has too few elements to reach the desired number of evaluation samples"
        sample_indices.extend(np.random.permutation(c_indices)[:n_samples_per_class])
    
    # Compute cross-entropy loss for each of the samples
    results = []
    for i in sample_indices:
        X_sample, Y_sample = X_train[i], Y_train[i]
        output = func(X_sample)
        results.append(np.argmax(output) == Y_sample)
    
    # Return the average cross-entropy loss
    return (np.average(results),)

In [40]:
# Add the fitness function to the toolbox
toolbox.register("evaluate", fitness, n_samples_to_test=N_SAMPLES_TO_TEST)

## Evolution components

##### Parent selection

In [41]:
# Redefine the tools.selDoubleTournament function for our multi-tree individuals (all comments from the original function have been removed to aid readability; the function remains unchanged 
# except for the usage of get_total_size instead of len (as well as some differences in imports and python version)
def selDoubleTournament(individuals, k, fitness_size, parsimony_size, fitness_first,cull_ratio = 0.3, fit_attr="fitness"):
    assert (1 <= parsimony_size <= 2), "Parsimony tournament size has to be in the range [1, 2]."

    def _sizeTournament(individuals, k, select):
        chosen = []
        for i in range(k):
            prob = parsimony_size / 2.
            ind1, ind2 = select(individuals, k=2)

            # This is the part that matters for our re-implementation: we use the total size of
            # all trees instead of the length of the individual, which is equal for all individuals
            if ind1.n_connections > ind2.n_connections:
                ind1, ind2 = ind2, ind1
            elif ind1.n_connections == ind2.n_connections:
                prob = 0.5

            chosen.append(ind1 if random.random() < prob else ind2)

        return chosen

    def _fitTournament(individuals, k, select):
        chosen = []
        for i in range(k):
            sorted_by_rank = select(individuals, k=len(individuals))
            number_to_cull = int(cull_ratio * len(individuals))
            aspirants = sorted_by_rank[:number_to_cull]
            pdb.set_trace()
            chosen.append(max(aspirants, key=attrgetter(fit_attr)))
        return chosen

    if fitness_first:
        tfit = partial(_fitTournament, select=tools.selNSGA2) #select=tools.selRandom)
        return _sizeTournament(individuals, k, tfit)
    else:
        tsize = partial(_sizeTournament, select=tools.selRandom)
        return _fitTournament(individuals, k, tsize)

In [42]:
# Define how to select parents (either a double tournament that controls for bloat or a single tournament that does not)
if SIZE_TOURNAMENT:
    toolbox.register("select", selDoubleTournament, fitness_size=TOURNAMENT_SIZE, 
                     parsimony_size=P_SMALLER_WINS*2, fitness_first=False)
else:
    toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)

##### Mutation

In [43]:
def mutate(tree):
    
    tree = copy.deepcopy(tree)
    
    def _remove_connection():
        possible_parents = [t for t in tree.outputs + tree.internals if len(t.children) > 1]
        if not possible_parents:
            return
        parent = random.choice(possible_parents)
        child_to_remove = random.choice(parent.children)
        parent.children.remove(child_to_remove)
        child_to_remove.parents.remove(parent)
        tree.n_connections -= 1
    
    def _add_connection():
        def _is_ancestor(node1, node2):
            return node1 in [node2] + node2.parents or any([_is_ancestor(node1, node3) for node3 in node2.parents])
        valid_child = False
        while not valid_child:
            parent = random.choice(tree.outputs + tree.internals)
            child = random.choice(tree.inputs + tree.internals)
            valid_child = not _is_ancestor(child, parent)
        child.parents.append(parent)
        parent.children.append(child)
        tree.n_connections += 1
        
    def _add_node():
        parent = random.choice(tree.outputs + tree.internals)
        children = random.sample(parent.children, random.randint(1, len(parent.children)))
        [parent.children.remove(child) for child in children]

        node = PrimitiveNode(random.choice(function_names))
        parent.children.append(node)
        node.parents.append(parent)
        node.children.extend(children)
        for child in children:
            child.parents.append(node)
        tree.internals.append(node)
        tree.n_connections += 1

    def _remove_node():
        if not tree.internals:
            return
        node = random.choice(tree.internals)
        for parent in node.parents:
            parent.children.extend(node.children)
        for child in node.children:
            child.parents.extend(node.parents)
        tree.internals.remove(node)
        connection_change = len(node.children) * (len(node.parents)-1) - len(node.parents)
        tree.n_connections += connection_change

    def _change_operator():
        if not tree.internals:
            return
        node = random.choice(tree.internals)
        node.name = random.choice(function_names)
        
    if random.random() < P_ADD_CONNECTION:
        _add_connection()
    if random.random() < P_REMOVE_CONNECTION:
        _remove_connection()
    if random.random() < P_ADD_NODE:
        _add_node()
    if random.random() < P_REMOVE_NODE:
        _remove_node()
    if random.random() < P_CHANGE_OPERATOR:
        _change_operator()
        
    return tree,

In [44]:
# Add the mutation ("mutate") function to the toolbox
toolbox.register("mutate", mutate)

##### Height boundary

In [45]:
# Define upper limits for height of trees (if limit is exceeded, a random parent is used instead)
if CONNECTIONS_CAPPED:
    toolbox.decorate("mutate", gp.staticLimit(key=lambda ind: ind.n_connections, max_value=MAX_CONNECTIONS))

## Defining statistics

In [46]:
# Describe which kinds of statistics to keep track of
stats_fit = tools.Statistics(key = lambda ind: ind.fitness.values)
stats_connections = tools.Statistics(key = lambda ind: ind.n_connections)
mstats = tools.MultiStatistics(fitness=stats_fit, connections=stats_connections)

In [47]:
# Describe metrics to keep track of for each statistic
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

## Running the genetic programming algorithm

In [48]:
pop = toolbox.population(POPULATION_SIZE)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(population=pop, toolbox=toolbox, cxpb=0.0, mutpb=1.0, 
                               ngen=N_GENERATIONS, stats=mstats, halloffame=hof, verbose=True)

   	      	                connections                	                     fitness                     
   	      	-------------------------------------------	-------------------------------------------------
gen	nevals	avg	gen	max	min	nevals	std	avg   	gen	max 	min 	nevals	std      
0  	100   	100	0  	100	100	100   	0  	0.1055	0  	0.21	0.04	100   	0.0353235
> <ipython-input-41-d91cbe7b32ef>(28)_fitTournament()
     26             aspirants = select(individuals, k=len(individuals))
     27             pdb.set_trace()
---> 28             chosen.append(max(aspirants, key=attrgetter(fit_attr)))
     29         return chosen
     30 

ipdb> cont


AttributeError: 'Individual' object has no attribute 'rank'

In [ ]:
# Store results
with open(RESULTS_FILENAME, "wb") as f:
    pickle.dump((pop, log, hof), f)

## Plotting statistics

In [ ]:
# Extract generation IDs, minimum fitnesses and average total heights per generation
gen = log.select("gen")
fitness_best = log.chapters["fitness"].select("max") 
conn_avg = log.chapters["connections"].select("avg")

In [ ]:
# Plot line for minimum fitness
fig, fit_ax = plt.subplots()
fit_line = fit_ax.plot(gen, fitness_best, "b-", label="Best Fitness")
fit_ax.set_xlabel("Generation")
fit_ax.set_ylabel(f"Accuracy", color="b")
for tl in fit_ax.get_yticklabels():
    tl.set_color("b")

# Plot line for average total height
height_ax = fit_ax.twinx()
height_line = height_ax.plot(gen, conn_avg, "r-", label="Average Number of Connections")
height_ax.set_ylabel("Average height", color="r")
for tl in height_ax.get_yticklabels():
    tl.set_color("r")

# Add legend
lines = fit_line + height_line
labs = [l.get_label() for l in lines]
fit_ax.legend(lines, labs, loc="upper center")

# Show the result
plt.show()

## Inspecting the best individual

In [ ]:
best_ind = hof[0]
print(f"Fitness of best individual: {best_ind.fitness}")

##### Printing trees

In [ ]:
# Print the trees of the best individual
for string in best_ind.get_strings():
    print(f"{string}\n")

##### Computing training & validation accuracy

In [ ]:
# Retrieving predictions from an individual
def get_predictions(individual, X):
    func = compile_multiclasstree(individual)
    predictions = []
    for i in range(X.shape[0]):
        outputs_i = func(X[i])
        predictions.append(np.argmax(outputs_i))
    return predictions

In [ ]:
# Retrieve predictions of the best individual on the training and validation sets
Y_train_pred = get_predictions(best_ind, X_train)
Y_test_pred = get_predictions(best_ind, X_test)

In [ ]:
# Compute accuracy of predictions
def compute_accuracy(Y_pred, Y_true):
    n_correct = np.sum(Y_pred == Y_true)
    return n_correct/Y_true.shape[0]

In [ ]:
# Print training and validation accuracies of the best individual
print(f"Training accuracy: {compute_accuracy(Y_train_pred, Y_train)}")
print(f"Validation accuracy: {compute_accuracy(Y_test_pred, Y_test)}")

##### Confusion matrices

In [ ]:
# Compute confusion matrices
cm_train = confusion_matrix(Y_train, Y_train_pred, labels=range(N_CLASSES_TO_USE))
cm_test = confusion_matrix(Y_test, Y_test_pred, labels=range(N_CLASSES_TO_USE))

In [ ]:
# Create figure
fig, ax = plt.subplots(1,2, figsize=(11,4))

# Plot confusion matrix for training data
sns.heatmap(cm_train, annot=True, fmt='g', ax=ax[0], cmap="Blues")
ax[0].set_xlabel('Predicted labels')
ax[0].set_ylabel('True labels')
ax[0].set_title('Confusion matrix for training data')

# Plot confusion matrix for validation data
sns.heatmap(cm_test, annot=True, fmt='g', ax=ax[1], cmap="Blues")
ax[1].set_xlabel('Predicted labels')
ax[1].set_ylabel('True labels')
ax[1].set_title('Confusion matrix for validation data')

# Show the result
plt.show()

##### Used features

In [ ]:
# Create plots of the inputs (pixels) used in the tree of the best individual for each class 
fig, ax = plt.subplots(1, N_CLASSES_TO_USE, figsize=(20,20/N_CLASSES_TO_USE))
for i, tree in enumerate(best_ind.get_strings()):
    inputs_used = list(map(int, re.findall("[0-9]+", str(tree))))
    input_tallies = np.zeros(X_train.shape[1])
    for arg in inputs_used:
        input_tallies[arg] += 1
    img_shape = int(math.sqrt(X_train.shape[1]))
    ax[i].imshow(input_tallies.reshape(img_shape, img_shape))
    ax[i].axis("off")
    ax[i].set_title(f"Pixels used for class {i}") 
plt.show()